In [1]:
import torch


In [2]:
N = 1
H = 28
W = 28
out_channels = 16
kernel_size = 5


In [3]:
from groups.discrete_so2 import DiscreteSO2
from models.layers.lifting_conv import LiftingConv

group = DiscreteSO2(order=4, device="cpu")
conv = LiftingConv(
    group=group,
    in_channels=1,
    out_channels=out_channels,
    kernel_size=kernel_size
)


In [5]:
signal = torch.rand((1, H, W))
print(conv(signal.view(1, 1, H, W)).shape)


torch.Size([1, 16, 4, 26, 26])


In [ ]:
rotated_signal_per_group_elem = group.left_action_on_r_n_signal(
    group.elements(), signal)
rotated_signal_per_group_elem.shape


torch.Size([4, 1, 28, 28])

In [ ]:
for i in range(len(group.elements())):
  with torch.no_grad():
    g = group.elements()[i]
    rotated_signal = rotated_signal_per_group_elem[i]

    rotated_signal_then_conv = conv(rotated_signal.view(
        1, *rotated_signal.shape)).squeeze(0)
    original_signal_conv = conv(signal.view(1, *signal.shape)).squeeze(0)
    original_signal_conv_then_rotation = group.left_action_on_group_signal(
        g.view(1, 1), original_signal_conv).squeeze(0)

    # check equivariance
    print(
        torch.allclose(rotated_signal_then_conv,
                       original_signal_conv_then_rotation, atol=1e-5, rtol=1e-5)
    )


True
True
True
True
